In [6]:
import numpy as np
import torch

0.030000000000000002

In [53]:

# 교란 라벨 
cluster = 10 # 레벨의 범위 0 ~ 9 
alpha = 30 # 교란 확률 ex) 원래 라벨이 30%확률로 다른 라벨로 교란됨
p_c = (1-((cluster-1)/cluster)*(alpha/100))  # 원래 라벨이 선택될 확률 값
p_i = (1-p_c)/(cluster-1) # 원래 라벨 외의 라벨이 선택될 확률 값

#y = np.random.choice(10, (2,5))
y = np.array([[3, 3, 0, 0, 4],[9, 5, 9, 2, 4]])
# (2,5) 데이터를 (10,1) 으로 변경한다.
y_tensor = torch.LongTensor(y).view(-1, 1)

#print(y_tensor)

depth = cluster # 라벨 인덱스의 범위 ex) 10 : 0~9

y_one_hot = torch.ones(y_tensor.size()[0], depth)*p_i # (10, 10) 형태의 ones 텐서를 만들고 p_i(기본확률값)값을 브로트캐스팅한다.
y_one_hot.scatter_(1, y_tensor, p_c) # tensor.scatter_(dim, index, val): 텐서의 dim차원에 index번째 원소에 val을 대입
#print(y_one_hot)
# *(tuple((2,5))+(-1,)) => 2 5 -1 , *: 언팩킹, 2 5 -1 값이 차원값으로 대입된다.
y_one_hot = y_one_hot.view(*(tuple(y.shape)+(-1,))) # 입력값의 차원이(2,5) 였다면 one-hot 인코딩 된 차원으로 변경하면 (2,5,10) 이된다.

distribution = torch.distributions.OneHotCategorical(y_one_hot) # Multinoulli 분포를 통해 
y_distributed = distribution.sample() # 입력된 확률에 의한 샘플을 뽑음 ex) [0.1, 0.1, 0.8] => [0, 0, 1]
#print(y_one_hot)
print(y_distributed) 
y_distributed.max(dim=y.ndim)[1]

tensor([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]])


tensor([[2, 3, 0, 6, 4],
        [9, 5, 9, 5, 4]])

In [70]:
# 노이즈 생성하기(회귀에서 사용)
label = torch.rand(5)
print(label)
noise = torch.normal(mean=0, std=1e-8, size=(len(label), 1))
#print(label)
#print(noise)
alpha = 0.2
noise_el_cnt = int(5*(1-alpha))
noise_el_idx = torch.randint(0, len(label), (noise_el_cnt,))
print(noise_el_cnt)
print(noise_el_idx)
noise[noise_el_idx] = 0
print(noise)



tensor([0.2838, 0.9976, 0.8301, 0.7734, 0.9808])
4
tensor([2, 4, 3, 0])
tensor([[ 0.0000e+00],
        [-2.0132e-08],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00]])


In [ ]:
# 라벨 스무딩